<a href="https://colab.research.google.com/github/antonellagambarte/procesamiento_del_lenguaje_I/blob/main/Desafio2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import platform
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import text_to_word_sequence

import multiprocessing
try:
  from gensim.models import Word2Vec
except:
  !pip install gensim
  from gensim.models import Word2Vec

from gensim.models.callbacks import CallbackAny2Vec

from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 66.1 MB/s eta 0:00:00


  Carga del dataset

In [2]:
# Descarga de carpeta de dataset
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

--2025-11-12 22:56:39--  http://songs_dataset.zip/
Resolving songs_dataset.zip (songs_dataset.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘songs_dataset.zip’
--2025-11-12 22:56:39--  https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip [following]
--2025-11-12 22:56:39--  https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.

In [3]:
os.listdir("./songs_dataset/")

['Kanye_West.txt',
 'lil-wayne.txt',
 'blink-182.txt',
 'eminem.txt',
 'bob-marley.txt',
 'michael-jackson.txt',
 'amy-winehouse.txt',
 'bruce-springsteen.txt',
 'alicia-keys.txt',
 'bob-dylan.txt',
 'ludacris.txt',
 'bruno-mars.txt',
 'rihanna.txt',
 'nursery_rhymes.txt',
 'prince.txt',
 'Lil_Wayne.txt',
 'cake.txt',
 'kanye.txt',
 'dickinson.txt',
 'lin-manuel-miranda.txt',
 'lorde.txt',
 'al-green.txt',
 'joni-mitchell.txt',
 'r-kelly.txt',
 'leonard-cohen.txt',
 'notorious-big.txt',
 'nickelback.txt',
 'adele.txt',
 'patti-smith.txt',
 'bjork.txt',
 'radiohead.txt',
 'janisjoplin.txt',
 'kanye-west.txt',
 'missy-elliott.txt',
 'bieber.txt',
 'britney-spears.txt',
 'jimi-hendrix.txt',
 'johnny-cash.txt',
 'beatles.txt',
 'drake.txt',
 'dr-seuss.txt',
 'dj-khaled.txt',
 'nirvana.txt',
 'notorious_big.txt',
 'paul-simon.txt',
 'dolly-parton.txt',
 'nicki-minaj.txt',
 'lady-gaga.txt',
 'disney.txt']

El artista elegido para realizar el desafío es Michael Jackson.

In [4]:
df = pd.read_csv('songs_dataset/michael-jackson.txt', sep='/n', header=None)
df.head()

/tmp/ipython-input-2822025462.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('songs_dataset/michael-jackson.txt', sep='/n', header=None)


,0
0,[Spoken Intro:]
1,You ever want something
2,that you know you shouldn't have
3,"The more you know you shouldn't have it,"
4,The more you want it


In [5]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 9484


## **Creación de vectores**

Preprocesamiento:

A continuación, se recorre cada línea del dataframe, se convierte la línea en una lista de palabras y se guarda todas esas listas en sentence_tokens.

sentence_tokens es una lista de listas, donde cada sublista son las palabras (tokens) de una línea de canción.

In [7]:
sentence_tokens = []
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

sentence_tokens[:2]

[['spoken', 'intro'], ['you', 'ever', 'want', 'something']]

Clase personalizada para registrar el loss

In [8]:
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

Modelo Skipgram

In [9]:
# Estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,# dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples
                     workers=1,
                     sg=1)           # modelo skipgram

Se construye el vocabulario con las palabras tokenizadas

In [10]:
w2v_model.build_vocab(sentence_tokens)

In [11]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 9484


In [12]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 1000


Entrenamiento de modelo con el callback personalizado

In [13]:
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=30,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 332195.875
Loss after epoch 1: 227160.0625
Loss after epoch 2: 226886.4375
Loss after epoch 3: 220579.8125
Loss after epoch 4: 191716.8125
Loss after epoch 5: 180949.5
Loss after epoch 6: 173434.75
Loss after epoch 7: 166188.5
Loss after epoch 8: 162249.75
Loss after epoch 9: 156011.625
Loss after epoch 10: 144920.125
Loss after epoch 11: 138371.5
Loss after epoch 12: 135980.5
Loss after epoch 13: 134118.75
Loss after epoch 14: 133386.75
Loss after epoch 15: 131142.0
Loss after epoch 16: 131250.75
Loss after epoch 17: 129227.25
Loss after epoch 18: 129950.25
Loss after epoch 19: 128118.0
Loss after epoch 20: 127296.75
Loss after epoch 21: 127572.5
Loss after epoch 22: 127668.75
Loss after epoch 23: 126239.0
Loss after epoch 24: 124603.5
Loss after epoch 25: 125834.5
Loss after epoch 26: 120929.5
Loss after epoch 27: 115384.0
Loss after epoch 28: 115261.0
Loss after epoch 29: 115863.5


(939805, 1490670)

## **Términos de interés**

In [14]:
# Palabras más similares
print("Similares a 'love':")
w2v_model.wv.most_similar(positive=["love"], topn=10)

Similares a 'love':


[('summer', 0.5443717837333679),
 ('rarest', 0.5350003242492676),
 ("makin'", 0.5349574089050293),
 ('farewell', 0.5268529653549194),
 ('power', 0.5205035209655762),
 ("fallin'", 0.5167802572250366),
 ('caress', 0.5139563083648682),
 ("livin'", 0.503252387046814),
 ('bear', 0.4954725503921509),
 ('stranger', 0.4929939806461334)]

In [15]:
print("\nSimilares a 'dance':")
w2v_model.wv.most_similar(positive=["dance"], topn=10)



Similares a 'dance':


[('floor', 0.7352413535118103),
 ('across', 0.673948347568512),
 ('rock', 0.6739422678947449),
 ('carpet', 0.6593192219734192),
 ('burn', 0.6519386768341064),
 ('land', 0.6425650119781494),
 ("speedin'", 0.6424461603164673),
 ('boogie', 0.6421562433242798),
 ('screen', 0.6388421654701233),
 ('sunlight', 0.6364737749099731)]

In [16]:
print("\nSimilares a 'baby':")
w2v_model.wv.most_similar(positive=["baby"], topn=10)



Similares a 'baby':


[('babe', 0.5374695062637329),
 ('factual', 0.5257635116577148),
 ('maria', 0.5047630071640015),
 ("beggin'", 0.5002064108848572),
 ('regret', 0.4972962737083435),
 ('won’t', 0.49674123525619507),
 ("cryin'", 0.4966360926628113),
 ('ooooooh', 0.49598267674446106),
 ('laughing', 0.49573245644569397),
 ('loves', 0.4880443215370178)]

In [17]:
# Palabras menos relacionadas
print("\nMenos similares a 'love':")
w2v_model.wv.most_similar(negative=["love"], topn=10)



Menos similares a 'love':


[('uuh', 0.0898258313536644),
 ('refrain', 0.07473364472389221),
 ('siedah', 0.03886207193136215),
 ('out', -0.001337222522124648),
 ('cheater', -0.002821546047925949),
 ('pitbull', -0.009847220033407211),
 ('stevie', -0.010197421535849571),
 ('people', -0.0446183942258358),
 ('lib', -0.05634492263197899),
 ('chorus', -0.06208831071853638)]

## **Reducción de dimensionalidad**

In [18]:
def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

vecs, labels = reduce_dimensions(w2v_model)

In [19]:
MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab